In [1]:
import torch

# Check if GPU is available
if torch.cuda.is_available():
    print("GPU is available.")
else:
    print("No GPU detected.")


GPU is available.


Thanks to https://github.com/thuml/Time-Series-Library for providing a repository with the implementations of Informer, Autoformer and DLinear. The repository is cloned here and slighty adopted.

In [2]:
# Clone the repository
!git clone https://github.com/thuml/Time-Series-Library.git

Cloning into 'Time-Series-Library'...
remote: Enumerating objects: 1210, done.
remote: Counting objects: 100% (749/749), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 1210 (delta 674), reused 659 (delta 659), pack-reused 461
Receiving objects: 100% (1210/1210), 78.00 MiB | 35.42 MiB/s, done.
Resolving deltas: 100% (868/868), done.


In [3]:
# Use the Time-Series-Library env and make a 'dataset' folder
%cd /content/Time-Series-Library
!mkdir dataset

/content/Time-Series-Library


Import Python librairies:

In [4]:
import pandas as pd
!pip install patool sktime reformer_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.8 MB/s eta 0:00:00
  Created wheel for axial-positional-embedding: filename=axial_positional_embedding-0.2.1-py3-none-any.whl size=2882 sha256=ed9995c58b88887f1ed3dc05e7778aabea7dcf7403a1e03f14ff56870460164f
  Stored in directory: /root/.cache/pip/wheels/b1/cb/39/7ce7ff2d2fd37cfe1fe7b3a3c43cf410632b2ad3b3f3986d73
Successfully built axial-positional-embedding


Import the data:

- firstly, import the df.csv in the "dataset" folder
- we will do univariate forecasting for the target "Price (EUR/MWhe)"
- data the TSLib models import must have the following format : [date, (eventually) predictors/features, target]
- we will create this dataframe to use from the df1.csv file that will be written in the "dataset" folder

In [9]:
df= pd.read_csv("dataset/df.csv")
#df=df[['Datetime (Local)','Price (EUR/MWhe)']]
#df['Datetime (Local)']=pd.to_datetime(df['Datetime (Local)'])
df['date'] = df['Datetime (Local)'].copy()
#df=df.set_index('Datetime (Local)')
#df=df[['Price (EUR/MWhe)']]

df=df.tail(10000)
#df
selected_columns = ['date', 'Consommation (MW)', 'Price (EUR/MWhe)']
df = df[selected_columns]
df.to_csv('/content/Time-Series-Library/dataset/df1.csv', index=False)

df
#df1.csv is to be used in the experiment


,date,Consommation (MW),Price (EUR/MWhe)
54990,2021-04-10 08:00:00,48387.5,66.20
54991,2021-04-10 09:00:00,51062.5,69.19
54992,2021-04-10 10:00:00,52583.5,64.51
54993,2021-04-10 11:00:00,53251.5,61.21
54994,2021-04-10 12:00:00,55236.0,59.37
...,...,...,...
64985,2022-05-31 19:00:00,48218.5,249.80
64986,2022-05-31 20:00:00,46607.5,254.67
64987,2022-05-31 21:00:00,44360.0,247.51
64988,2022-05-31 22:00:00,44543.0,240.01


Run experiments:
Feature = S: it means we do univariate forecasting. No other features allowed
We keep the original parameters. Change onnly pred_len (12,24, 36 or 48 in our experiment)

In [25]:
model_name = "Autoformer"

!python -u run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path ./dataset/ \
  --data_path df1.csv \
  --model_id df_01_Autoformer \
  --model $model_name \
  --data custom \
  --features S \
  --target "Price (EUR/MWhe)" \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 48 \
  --e_layers 2 \
  --d_layers 5 \
  --factor 5 \
  --enc_in 1 \
  --dec_in 1 \
  --c_out 1 \
  --des 'Exp' \
  --itr 1 \
  --train_epochs 2


Args in experiment:
Basic Config
  Task Name:          long_term_forecast  Is Training:        1                   
  Model ID:           df_01_Autoformer    Model:              Autoformer          

Data Loader
  Data:               custom              Root Path:          ./dataset/          
  Data Path:          df1.csv             Features:           S                   
  Target:             Price (EUR/MWhe)    Freq:               h                   
  Checkpoints:        ./checkpoints/      

Forecasting Task
  Seq Len:            96                  Label Len:          48                  
  Pred Len:           48                  Seasonal Patterns:  Monthly             
  Inverse:            0                   

Model Parameters
  Top k:              5                   Num Kernels:        6                   
  Enc In:             1                   Dec In:             1                   
  C Out:              1                   d model:            512                 
  

In [14]:
model_name = "DLinear"

!python -u run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path ./dataset/ \
  --data_path df1.csv \
  --model_id df_01_DLinear \
  --model $model_name \
  --data custom \
  --features S \
  --target "Price (EUR/MWhe)" \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 48 \
  --e_layers 2 \
  --d_layers 5 \
  --factor 5 \
  --enc_in 1 \
  --dec_in 1 \
  --c_out 1 \
  --des 'Exp' \
  --itr 1 \
  --train_epochs 2

Args in experiment:
Basic Config
  Task Name:          long_term_forecast  Is Training:        1                   
  Model ID:           df_01_DLinear       Model:              DLinear             

Data Loader
  Data:               custom              Root Path:          ./dataset/          
  Data Path:          df1.csv             Features:           S                   
  Target:             Price (EUR/MWhe)    Freq:               h                   
  Checkpoints:        ./checkpoints/      

Forecasting Task
  Seq Len:            96                  Label Len:          48                  
  Pred Len:           48                  Seasonal Patterns:  Monthly             
  Inverse:            0                   

Model Parameters
  Top k:              5                   Num Kernels:        6                   
  Enc In:             1                   Dec In:             1                   
  C Out:              1                   d model:            512                 
  

In [18]:
model_name = "Informer"

!python -u run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path ./dataset/ \
  --data_path df1.csv \
  --model_id df_01_Informer \
  --model $model_name \
  --data custom \
  --features S \
  --target "Price (EUR/MWhe)" \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 48 \
  --e_layers 2 \
  --d_layers 5 \
  --factor 5 \
  --enc_in 1 \
  --dec_in 1 \
  --c_out 1 \
  --des 'Exp' \
  --itr 1 \
  --train_epochs 2

Args in experiment:
Basic Config
  Task Name:          long_term_forecast  Is Training:        1                   
  Model ID:           df_01_Informer      Model:              Informer            

Data Loader
  Data:               custom              Root Path:          ./dataset/          
  Data Path:          df1.csv             Features:           S                   
  Target:             Price (EUR/MWhe)    Freq:               h                   
  Checkpoints:        ./checkpoints/      

Forecasting Task
  Seq Len:            96                  Label Len:          48                  
  Pred Len:           48                  Seasonal Patterns:  Monthly             
  Inverse:            0                   

Model Parameters
  Top k:              5                   Num Kernels:        6                   
  Enc In:             1                   Dec In:             1                   
  C Out:              1                   d model:            512                 
  

In [22]:
model_name = "iTransformer"

!python -u run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path ./dataset/ \
  --data_path df1.csv \
  --model_id df_01_iTransformer \
  --model $model_name \
  --data custom \
  --features S \
  --target "Price (EUR/MWhe)" \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 48 \
  --e_layers 2 \
  --d_layers 5 \
  --factor 5 \
  --enc_in 1 \
  --dec_in 1 \
  --c_out 1 \
  --des 'Exp' \
  --itr 1 \
  --train_epochs 2

Args in experiment:
Basic Config
  Task Name:          long_term_forecast  Is Training:        1                   
  Model ID:           df_01_iTransformer  Model:              iTransformer        

Data Loader
  Data:               custom              Root Path:          ./dataset/          
  Data Path:          df1.csv             Features:           S                   
  Target:             Price (EUR/MWhe)    Freq:               h                   
  Checkpoints:        ./checkpoints/      

Forecasting Task
  Seq Len:            96                  Label Len:          48                  
  Pred Len:           48                  Seasonal Patterns:  Monthly             
  Inverse:            0                   

Model Parameters
  Top k:              5                   Num Kernels:        6                   
  Enc In:             1                   Dec In:             1                   
  C Out:              1                   d model:            512                 
  